In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils
from keras.models import Sequential
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

In [ ]:
path = '/content/drive/My Drive/Garbage classification'
trash = {'glass':0, 'metal':1, 'paper':2, 'plastic':3, 'plastic_bag':4}

In [ ]:
trainDataGen = ImageDataGenerator(rescale=1./255, rotation_range = 30, width_shift_range=0.2, height_shift_range=0.2,
                                 shear_range=0.2, zoom_range=0.2, horizontal_flip=False, vertical_flip=False, fill_mode='nearest')

trainGenSet = trainDataGen.flow_from_directory(
    path + '/train',
    batch_size=32,
    target_size=(224, 224),
    class_mode='categorical')

Found 1700 images belonging to 5 classes.


In [ ]:
print(trainGenSet.class_indices)
print(testGenSet.class_indices)

#for f in testGenSet.filepaths:
  #print(f)

{'glass': 0, 'metal': 1, 'paper': 2, 'plastic': 3, 'plastic_bag': 4}
{'glass': 0, 'metal': 1, 'paper': 2, 'plastic': 3, 'plastic_bag': 4}


In [ ]:
testDataGen = ImageDataGenerator(rescale=1./255)
testGenSet = testDataGen.flow_from_directory(
    path + '/test',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 300 images belonging to 5 classes.


In [ ]:
VGG16_MODEL = tf.keras.applications.VGG16(input_shape = (224, 224, 3),include_top = False, weights = 'imagenet')
VGG16_MODEL.trainable=True
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
#flatten_layer = tf.keras.layers.Flatten()
prediction_layer = tf.keras.layers.Dense(5, activation='softmax')

model = tf.keras.Sequential([VGG16_MODEL, global_average_layer, prediction_layer])
#model = VGG16_MODEL
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
from tensorflow.keras.models import Model

VGG16_MODEL = tf.keras.applications.VGG16(input_shape = (224, 224, 3),include_top = False, weights = 'imagenet')
VGG16_MODEL.trainable=True
last = VGG16_MODEL.output

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(last)
flatten_layer = tf.keras.layers.Flatten()(global_average_layer)
prediction_layer = tf.keras.layers.Dense(5, activation='softmax')(flatten_layer)

model = Model(VGG16_MODEL.input, prediction_layer)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), strides = 1, padding='same', input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))

model.add(Conv2D(128, kernel_size=(3,3), strides = 1, padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=(3,3), strides = 1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=(3,3), strides = 1, padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(3,3), strides = 1, padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(3,3), strides = 1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(Dropout(0.25))

model.add(Conv2D(512, kernel_size=(3,3), strides = 1, padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3,3), strides = 1, padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3,3), strides = 1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))

model.add(Conv2D(512, kernel_size=(3,3), strides = 1, padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3,3), strides = 1, padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3,3), strides = 1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 120, 120, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 120, 120, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 60, 60, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 60, 60, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 128)       0

In [ ]:
model = tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3))

for layer in model.layers[:-10]:
    layer.trainable = False

for layer in model.layers:
    print(layer, layer.trainable)

top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dense(5, activation='softmax'))

top_model.load_weights(top_model_weights_path) #이미 학습된 Flatten()을 불러와야함! 반드시

new_model = Sequential() # VGG16(fine-tuning) + Faltten 모델(이미 학습됨)을 합쳐서 모델생성
for l in model.layers:
    new_model.add(l)

for l in top_model.layers:
    new_model.add(l)

In [ ]:
#VGG16모델을 학습하지 않고 flatten()같은 밑부분 모델만 학습시킨후, 이 가중치를 저장
#모델을 학습할 때는 저장한 가중치를 가져와서 사용.

In [ ]:
from keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2, preprocess_input, decode_predictions
#from keras.applications.resnet50 import preprocess_input, decode_predictions
#include_top을 false로 하면 convolution layer만 가져오고 fully connected layer를 쌓을 수 있음
resnet = ResNet152V2(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
x = resnet.output
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', input_dim = (150, 150, 3))(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(5, activation='softmax')(x)

model = Model(inputs=resnet.input, outputs=x)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x) #1024
predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy']) #rmsprop

In [ ]:
history = model.fit(trainGenSet, epochs=110, validation_steps=10, validation_data=testGenSet)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = 'upper left')
plt.show()

In [ ]:
#model evaluate
scores = model.evaluate(testGenSet, steps=1)
print("%.2f%%" %(scores[1]*100))
#model predict
output = model.predict(testGenSet, steps=1)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(output)

1/1 [==============================] - 1s 632ms/step - loss: 0.0014 - accuracy: 1.0000
100.00%
[[0.000 0.000 0.000 0.000 1.000]
 [0.000 0.000 0.000 0.000 1.000]
 [0.000 0.000 1.000 0.000 0.000]
 [1.000 0.000 0.000 0.000 0.000]
 [0.000 1.000 0.000 0.000 0.000]]


In [ ]:
from keras.preprocessing import image

example_dir = ['/test/glass/glass416.jpg', '/test/glass/glass432.jpg', '/test/metal/metal330.jpg', '/test/metal/metal385.jpg', '/test/paper/paper455.jpg', '/test/paper/paper493.jpg', '/test/plastic/plastic262.jpg', '/test/plastic/plastic437.jpg', '/test/plastic_bag/plastic_bag1316.jpg', '/test/plastic_bag/plastic_bag21.jpg']

for i in example_dir:
  print(i)
  img = image.load_img(path + i, target_size = (244, 244))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  output = model.predict(img)
  print(output)

In [ ]:
model.save('recycle_classification_inception_2.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# Save the model.
with open('model_unquant.tflite', 'wb') as f:
  f.write(tflite_model)